In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10) 
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + input)), negative_slope=0.1)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
   ConvTranspose2d-5            [-1, 256, 8, 8]       1,179,904
       BatchNorm2d-6            [-1, 256, 8, 8]             512
   ConvTranspose2d-7          [-1, 128, 16, 16]         295,040
       BatchNorm2d-8          [-1, 128, 16, 16]             256
   ConvTranspose2d-9            [-1, 2, 32, 32]           2,306
      BatchNorm2d-10            [-1, 2, 32, 32]               4
Total params: 2,662,278
Trainable params: 2,662,278
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 6.78
Params size (MB): 10.16
Estima

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00191282 (0.02437558), PSNR 27.18325043 (20.36489614), SSIM 0.78981578 (0.57827897)
Finished training epoch 0
Validate: MSE 0.00317362 (0.00273076), PSNR 24.98444366 (25.71257640), SSIM 0.70479381 (0.77342306)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00256411 (0.00262434), PSNR 25.91063309 (25.83511683), SSIM 0.76479244 (0.77170202)
Finished training epoch 1
Validate: MSE 0.00302899 (0.00267502), PSNR 25.18702698 (25.80275012), SSIM 0.70581925 (0.77280521)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00216536 (0.00257227), PSNR 26.64470482 (25.91678104), SSIM 0.79266030 (0.77135893)
Finished training epoch 2
Validate: MSE 0.00294707 (0.00260408), PSNR 25.30609322 (25.91477368), SSIM 0.70760971 (0.77411481)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00251951 (0.00248390), PSNR 25.98683929 (26.07137225), SSIM 0.79057282 (0.77157303)
Finished training epoch 3
Validate: MSE 0.00290729 (0.0

Epoch: 30, MSE 0.00221706 (0.00223875), PSNR 26.54223061 (26.52324980), SSIM 0.77523261 (0.78196735)
Finished training epoch 30
Validate: MSE 0.00257642 (0.00235136), PSNR 25.88983917 (26.34795770), SSIM 0.72465062 (0.78429485)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00191677 (0.00223736), PSNR 27.17429352 (26.52466435), SSIM 0.80466068 (0.78184172)
Finished training epoch 31
Validate: MSE 0.00288971 (0.00233626), PSNR 25.39144897 (26.37661748), SSIM 0.72087121 (0.78410785)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00228075 (0.00222699), PSNR 26.41922569 (26.54880278), SSIM 0.77134001 (0.78205221)
Finished training epoch 32
Validate: MSE 0.00285544 (0.00250045), PSNR 25.44327164 (26.08163104), SSIM 0.71818978 (0.78297465)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00143330 (0.00223335), PSNR 28.43662262 (26.53292914), SSIM 0.80583656 (0.78200947)
Finished training epoch 33
Validate: MSE 0.00301131 (0.00271577), PSNR 

Epoch: 60, MSE 0.00172287 (0.00216913), PSNR 27.63747978 (26.66079209), SSIM 0.80645829 (0.78297191)
Finished training epoch 60
Validate: MSE 0.00259997 (0.00231640), PSNR 25.85031891 (26.41481791), SSIM 0.72729290 (0.78571658)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00203476 (0.00216387), PSNR 26.91486359 (26.66967210), SSIM 0.76890832 (0.78315632)
Finished training epoch 61
Validate: MSE 0.00251865 (0.00226247), PSNR 25.98832893 (26.51118409), SSIM 0.72712553 (0.78175220)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00232394 (0.00216561), PSNR 26.33774567 (26.66443848), SSIM 0.78156620 (0.78300428)
Finished training epoch 62
Validate: MSE 0.00265709 (0.00224989), PSNR 25.75593185 (26.53410209), SSIM 0.72367907 (0.78158992)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00236165 (0.00216286), PSNR 26.26784897 (26.67133515), SSIM 0.78663266 (0.78300242)
Finished training epoch 63
Validate: MSE 0.00261061 (0.00244020), PSNR 

Epoch: 90, MSE 0.00188536 (0.00213834), PSNR 27.24604416 (26.72231351), SSIM 0.79251260 (0.78292347)
Finished training epoch 90
Validate: MSE 0.00253035 (0.00219735), PSNR 25.96819496 (26.63953715), SSIM 0.72630286 (0.78419613)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00177557 (0.00213068), PSNR 27.50662041 (26.73755109), SSIM 0.79116470 (0.78291994)
Finished training epoch 91
Validate: MSE 0.00263151 (0.00228277), PSNR 25.79794121 (26.47541814), SSIM 0.71820891 (0.78320996)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00225386 (0.00213132), PSNR 26.47072220 (26.73615606), SSIM 0.77696908 (0.78306159)
Finished training epoch 92
Validate: MSE 0.00245734 (0.00227907), PSNR 26.09534454 (26.48120079), SSIM 0.72351426 (0.77865221)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00236186 (0.00212861), PSNR 26.26745415 (26.74118786), SSIM 0.78056061 (0.78295953)
Finished training epoch 93
Validate: MSE 0.00250121 (0.00230895), PSNR 

Epoch: 120, MSE 0.00191111 (0.00210631), PSNR 27.18713760 (26.78600297), SSIM 0.78966969 (0.78280367)
Finished training epoch 120
Validate: MSE 0.00255619 (0.00222451), PSNR 25.92406273 (26.58366246), SSIM 0.72331655 (0.77825962)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00207090 (0.00210524), PSNR 26.83841705 (26.78625746), SSIM 0.77575111 (0.78275445)
Finished training epoch 121
Validate: MSE 0.00264347 (0.00221897), PSNR 25.77825928 (26.59864053), SSIM 0.71930671 (0.77951202)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00189245 (0.00211038), PSNR 27.22975159 (26.77461720), SSIM 0.79246151 (0.78255645)
Finished training epoch 122
Validate: MSE 0.00253796 (0.00226190), PSNR 25.95515060 (26.50753432), SSIM 0.72286165 (0.77708042)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00183855 (0.00210620), PSNR 27.35523415 (26.78768345), SSIM 0.79214227 (0.78264685)
Finished training epoch 123
Validate: MSE 0.00257847 (0.00227

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00256061 (0.00219704), PSNR 25.91656685 (26.64345032), SSIM 0.72719914 (0.78596641)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()